# 1.2 Clusters with age between 500 Myrs and 1 Gyrs

In [20]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from sklearn.neighbors import NearestNeighbors
import scipy as sp
from scipy.spatial import distance
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_scatter_density 
from matplotlib.colors import LinearSegmentedColormap
from scipy.optimize import curve_fit
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings('ignore')

## Functions

In [21]:
def preprocess_cluster(data, g_mean_th=18):
    """
    ------
    parallax > 0 

    phot_g_mean_mag < g_mean_th
    ------
    """
    data = data[data['parallax'] > 0]
    data = data[data['phot_g_mean_mag'] < g_mean_th]
    data['Gmg'] = data['phot_g_mean_mag'] + (5 * np.log10(data['parallax']) - 10)
    data['L'] = 10**(0.4*(4.83 - data['Gmg']))
    print(len(data))
    return data


def cmd_plot(data, x_axis, y_axis, alpha=0.8, s=5):
    """
    -------
    plot isochrone
    -------
    """

    with plt.style.context(['ieee']):
        fig = plt.figure(figsize=(6,6), dpi=100)
        sns.scatterplot(data=data, y=y_axis, x=x_axis, alpha=alpha, s=s)
        plt.gca().invert_yaxis();
        
        
def joint_plot(data):

    plt.figure(dpi=90)
    sns.jointplot(
        data=data,
        x="pmra", y="pmdec",
        kind="kde"
        );
    
    
def fit_curve(data, column, bins = 100):

    plt.figure(figsize=(12,3), dpi=120)
    counts, bins, patches = plt.hist(data[column], bins = bins)

    # Define the Gaussian function
    def gaussian(x, amp, mu, sigma):
        return amp * np.exp(-(x - mu)**2 / (2 * sigma**2))
    
    # data
    x_data = bins[:-1]
    y_data = counts
    
    # Fit the Gaussian function to the data
    popt, pcov = curve_fit(gaussian, x_data, y_data)
    
    plt.figure(figsize=(12,3), dpi=120)
    # Plot the original data and the fitted curve
    sns.scatterplot(x_data, y_data, label=column)
    plt.plot(x_data, gaussian(x_data, *popt), color='red', label='Fit')
    plt.legend()
    plt.show()
    
    return popt


def guassian_filter(data, column, mu, std):    
    up = round(mu + 3 * std, 2)
    low = round(mu - 3 * std, 2)
    
    if up > low:
        print('upper bound:', up) 
        print('lower bound:', low)
        df = data[(data[column] < up) & (data[column] > low)]
    else:
        print('upper bound:', low) 
        print('lower bound:', up)
        df = data[(data[column]< low) & (data[column] > up)]
    
    print('cluster length:', len(df))
    return df


def luminosity_density(cluster_3d, clusterdf):
    """
    --------
    cluster_3d --> measuring distance in that dataframe
    
    clusterdf --> main dataframe
    -------
    """
    # create a NearestNeighbors object and fit the dataset
    nbrs = NearestNeighbors(n_neighbors=6, metric='minkowski').fit(cluster_3d)

    # find the 5 nearest neighbors for each data point including itself
    distances, indices = nbrs.kneighbors(cluster_3d)

    # find maximum distance among 5 neighbors
    max_distances = np.amax(distances, axis=1)

    # sphere of that max distance
    spheres = (4/3) * np.pi * (max_distances ** 3)

    # sum of luminosities of each 6 nn
    lum_sum = []
    for i in range(len(clusterdf)):
        lum_sum.append(np.sum(clusterdf.iloc[indices[i]]['L']))

    # luminosity density
    lum_dens = lum_sum / spheres

    return lum_dens



def lum_plot(data):
    """
    --------
    plot for luminosity density profile
    --------
    """    
    
    with plt.style.context(['ieee']):
        plt.figure(figsize=(12,6), dpi=200)
        plt.plot(range(len(data)), np.sort(data))
        plt.ylabel('ΔL/ΔV')
        plt.title('luminosity density profile');

## open cluster names

In [23]:
open_clusters = pd.read_excel('../../data/open clusters table.xlsx')

open_clusters = open_clusters.dropna(axis=0, subset=['Name', 'logt']).drop(index=0)
open_clusters['logt'] = open_clusters.logt.astype(float)

x = open_clusters[(10**open_clusters['logt'] < 1 * 10**9) & (10**open_clusters['logt'] >= 0.5 * 10**9)].index
open_clusters.loc[x].dropna(axis=1)

,Name,RA,DEC,l,b,d,logt,D
1,Alessi 1,00 53 27,+49 34 11,123.26,-13.3,800,8.85,54
3,Alessi 3,07 16 19,-46 36 36,257.84,-15.37,288,8.87,66
8,Alessi 10,20 04 44,-10 28 47,31.64,-20.98,513,8.71,20.4
10,Alessi 13,03 28 12,-35 54 00,237.65,-55.71,110,8.72,384
15,Andrews-Lindsay 1,13 15 16,-65 55 18,305.4,-3.2,16900,8.90,1.5
...,...,...,...,...,...,...,...,...
893,Stock 4,01 52 58,+57 04 11,131.25,-4.81,1200,8.70,25.2
904,Stock 21,00 29 49,+57 55 11,120.05,-4.83,1100,8.72,16.8
933,Trumpler 31,17 59 49,-28 10 00,2.25,-2.29,986,8.87,5
949,vdB-Hagen 66,09 25 18,-54 43 00,276,-3.01,7000,8.90,4


In [26]:
open_clusters.loc[x].dropna(axis=1).tail(55)

,Name,RA,DEC,l,b,d,logt,D
374,King 7,03 59 00,+51 48 00,149.77,-1.02,2200,8.800,7
396,Loden 480,11 55 58,-58 24 00,295.7,3.69,1200,8.710,18
411,Loden 2313,15 40 48,-52 28 47,327.16,2.24,1410,8.740,22.8
418,Lynga 11,16 38 09,-46 19 00,338.18,0.45,2300,8.800,4.5
459,NGC 1245,03 14 42,+47 14 12,146.65,-8.93,2876,8.704,9
464,NGC 1496,04 04 32,+52 39 42,149.85,0.19,1230,8.800,4
476,NGC 1708,05 03 25,+52 50 59,155.74,6.85,600,8.760,25.2
487,NGC 1896,05 25 40,+29 17 59,177.34,-3.48,820,8.800,22.8
488,NGC 1901,05 18 15,-68 26 12,279.02,-33.6,415,8.920,15
512,NGC 2202,06 16 51,+06 00 00,203.62,-4.9,900,8.740,14.4
